In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization

num_layers = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate sinusoidal time series data
def generate_sin_wave(freq, num_samples, sample_rate):
    x = np.linspace(0, num_samples / sample_rate, num_samples)
    y = np.sin(2 * np.pi * freq * x)
    return y

# Create input-target pairs for the LSTM model
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length - 1):
        seq = data[i : (i + seq_length)]
        X.append(seq)
        target = data[i + seq_length]
        y.append(target)
    return np.array(X), np.array(y)

# Define the LSTM model with dropout
class TimeSeriesLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(TimeSeriesLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Generate sinusoidal time series data
freq = 2
num_samples = 200
sample_rate = 10
data = generate_sin_wave(freq, num_samples, sample_rate)

# Prepare the data for the LSTM model
seq_length = 20
X, y = create_sequences(data, seq_length)
X = X.reshape(X.shape[0], X.shape[1], 1)
y = y.reshape(y.shape[0], 1)

# Function to train and validate the model
def train_and_validate(X_train, y_train, X_val, y_val, num_epochs, device, hidden_size, learning_rate, weight_decay, dropout_prob):
    hidden_size = int(hidden_size)
    input_size = X_train.shape[2]
    output_size = y_train.shape[1]
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

    model = TimeSeriesLSTM(input_size, hidden_size, num_layers, output_size, dropout_prob=dropout_prob).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = criterion(val_outputs, y_val_tensor)
            val_losses.append(val_loss.item())

    return train_losses, val_losses

# Create a function to optimize using Bayesian optimization
def optimize_lstm(hidden_size, learning_rate, weight_decay, dropout_prob):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_epochs = 5
    k_folds = 5
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    val_losses_all = []

    for fold, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]

        _, val_losses = train_and_validate(X_train, y_train, X_val, y_val, num_epochs, device, hidden_size, learning_rate, weight_decay, dropout_prob)
        val_losses_all.append(val_losses)

    # Return the mean validation loss across all folds
    return -np.mean([losses[-1] for losses in val_losses_all])

# Define the bounds for hyperparameters
pbounds = {
    "hidden_size": (32, 256),
    "learning_rate": (1e-4, 1e-2),
    "weight_decay": (1e-5, 1e-3),
    "dropout_prob": (0.0, 0.5),
}

# Create the Bayesian optimization object
optimizer = BayesianOptimization(
    f=optimize_lstm,
    pbounds=pbounds,
    random_state=42,
)

# Perform Bayesian optimization
optimizer.maximize(init_points=5, n_iter=25)

# Print the best hyperparameters
print(optimizer.max)

# Perform 5-fold cross-validation
num_epochs = 5
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

train_losses_all = []
val_losses_all = []

for fold, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]

    train_losses, val_losses = train_and_validate(X_train, y_train, X_val, y_val, num_epochs, device)
    train_losses_all.append(train_losses)
    val_losses_all.append(val_losses)

    print(f"Fold {fold + 1}: Final Training Loss: {train_losses[-1]:.4f}, Final Validation Loss: {val_losses[-1]:.4f}")

# Plot training and validation losses
plt.figure(figsize=(12, 6))
for i in range(k_folds):
    plt.plot(range(1, num_epochs + 1), train_losses_all[i], label=f"Training Loss (Fold {i + 1})")
    plt.plot(range(1, num_epochs + 1), val_losses_all[i], label=f"Validation Loss (Fold {i + 1})")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()



|   iter    |  target   | dropou... | hidden... | learni... | weight... |
-------------------------------------------------------------------------
| 1         | -0.5128   | 0.1873    | 245.0     | 0.007347  | 0.0006027 |
| 2         | -0.5131   | 0.07801   | 66.94     | 0.000675  | 0.0008675 |
| 3         | -0.5096   | 0.3006    | 190.6     | 0.0003038 | 0.0009702 |
| 4         | -0.5176   | 0.4162    | 79.56     | 0.0019    | 0.0001916 |
| 5         | -0.5042   | 0.1521    | 149.5     | 0.004376  | 0.0002983 |
| 6         | -0.5037   | 0.2182    | 162.3     | 0.005269  | 0.0003376 |
| 7         | -0.5059   | 0.0       | 32.0      | 0.007962  | 0.0007266 |
| 8         | -0.5133   | 0.1011    | 161.1     | 0.00362   | 0.0006121 |
| 9         | -0.5081   | 0.4854    | 191.0     | 0.009939  | 0.0003802 |
| 10        | -0.5037   | 0.2634    | 162.9     | 0.005371  | 0.0001136 |
| 11        | -0.5045   | 0.2694    | 150.7     | 0.006583  | 0.000379  |
| 12        | -0.5096   | 0.4174    | 

TypeError: train_and_validate() missing 4 required positional arguments: 'hidden_size', 'learning_rate', 'weight_decay', and 'dropout_prob'